In [1]:
import os

In [2]:
%pwd

'e:\\2025\\Project_Learning\\NLP_Text_Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\2025\\Project_Learning\\NLP_Text_Summarizer'

In [6]:
from dataclasses import dataclass
from pathlib import Path

# The @dataclass(frozen=True) decorator automatically generates methods like __init__, __repr__, etc.
# frozen=True makes the instances immutable (read-only) after creation.
@dataclass(frozen=True)
class DataIngestionConfig:
    """
    Defines the structure for Data Ingestion configuration.
    The types match the values read from the config.yaml file.
    """
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories


In [8]:

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        # Load configurations using the read_yaml utility
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Ensure the main artifacts root directory exists
        create_directories([self.config.artifacts_root])

    # Method to get the Data Ingestion specific configuration
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir]) # Create ingestion root directory

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [29]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size
from pathlib import Path

# Assuming DataIngestionConfig is imported or defined earlier
# Example: from textSummarizer.entity import DataIngestionConfig 
# (You would need to ensure this import is present in your actual file)

class DataIngestion:
    """
    A component class to handle the data downloading and storage.
    """
    def __init__(self, config: DataIngestionConfig):
        # Store the immutable configuration object
        self.config = config

    def download_file(self):
        """
        Downloads the file from the source URL if it doesn't already exist locally.
        """
        # Check if the local data file path exists
        if not os.path.exists(self.config.local_data_file):
            
            # If it doesn't exist, download the file using urllib.request.urlretrieve
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            # If the file already exists, log its size
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    # CORRECTED: This method is now properly indented within the DataIngestion class.
    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory.
        """
        # Get the target directory for unzipping from the configuration
        unzip_path = self.config.unzip_dir
        
        # Ensure the destination directory exists
        os.makedirs(unzip_path, exist_ok=True)
        
        # Open the local zip file and extract all contents to the unzip_path
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            
        logger.info(f"Extracted zip file to: {unzip_path}")

In [35]:
import os

# Use the exact path from your configuration
file_to_delete = "artifacts/data_ingestion/data.zip" 

if os.path.exists(file_to_delete):
    os.remove(file_to_delete)
    print(f"✅ Removed corrupted file: {file_to_delete}")
else:
    print(f"File not found at {file_to_delete}. Proceeding with download.")

✅ Removed corrupted file: artifacts/data_ingestion/data.zip


In [36]:
try:
    # 1. Initialize Configuration Manager
    config = ConfigurationManager()
    
    # 2. Get the specific configuration for Data Ingestion
    data_ingestion_config = config.get_data_ingestion_config()
    
    # 3. Initialize the Data Ingestion Component
    data_ingestion = DataIngestion(config=data_ingestion_config)
    
    # 4. Execute the Data Ingestion steps
    # This must download the file completely!
    data_ingestion.download_file() 
    
    # This will now attempt to extract the newly downloaded file
    data_ingestion.extract_zip_file()
    
except Exception as e:
    raise e

[2025-10-28 10:44:30,210: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-28 10:44:30,211: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-28 10:44:30,213: INFO: common: created directory at: artifacts]
[2025-10-28 10:44:30,213: INFO: common: created directory at: artifacts/data_ingestion]
[2025-10-28 10:44:33,668: INFO: 1799174034: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 0F70:2A467C:A2B7F:172F29:69005136
Accept-Ranges: bytes
Date: Tue, 28 Oct 2025 05:14:31 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10246-MAA
X-Cache: MI